In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.nn.functional import one_hot

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:02<00:00, 72909501.37it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Fourier Transform function for images
def apply_fourier(x):
    # Moving the color channel to the last dimension to apply fft
    x = x.permute(0, 2, 3, 1)
    # Apply FFT, shift zero to center, then take absolute value (spectrum)
    f = torch.fft.fftn(x, dim=(1, 2))
    fshift = torch.fft.fftshift(f, dim=(1, 2))
    spectrum = torch.abs(fshift)
    return spectrum.permute(0, 3, 1, 2)  # Move the color channel back

In [ ]:
# Neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = apply_fourier(data).to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [ ]:
# Testing the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = apply_fourier(data).to(device)
            target = target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [ ]:
# Run training and testing
for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/50000 (0%)]	Loss: 1.340671
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.452029
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.408006
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.379966
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.350098
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.214327
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.315626
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.378917

Test set: Average loss: 0.0235, Accuracy: 4783/10000 (48%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.186804
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.350605
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.392923
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.408709
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.561475
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.211365
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.395809
Train Epoch: 2 [44800/50000 (90%)]	Loss: 1.244607

Test set: Average loss: 0.0235, Accuracy: 4735/10000 (47%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 1.519280
Train Epoch: 3 [6400/50000 (13%)]	Loss: 1.486